In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def getW(K,d):
    w = np.random.randn(K,d,d)
    return w

def getb(K,d):
    b = np.random.randn(K,d,1)
    return b

def getomega(d):
    omega = np.random.randn(d,1)
    return omega

def getmu():
    mu = np.random.randn(1)
    return mu


def getdJdmu(Z, ypsilon, c, omega, mu, K, eta_div):
    u = np.transpose(Z[K-1])@omega + mu
    
    dJdmu = np.transpose(eta_div(u))@(ypsilon-c)

    return dJdmu

def getdJdomega(Z, ypsilon, c, omega, mu, K, eta_div):
    u = np.transpose(Z[K-1])@omega + mu
    dJdOmega = Z[K-1] @ ((ypsilon - c) * eta_div(u))
    return dJdOmega

def getdJdW(P, Z, W, b, K, h, sigma_div):
    dJdW = np.zeros_like(W)
    for k in range(K):
        u = W[k]@Z[k]+b[k]
        dJdW[k] = h*(P[k+1]*sigma_div(u))@ np.transpose(Z[k])  
    return dJdW

def getdJdb(P, Z, W, b, K, h, sigma_div):
    dJdb = np.zeros_like(b)
    for k in range(K):
        u = W[k]@Z[k]+b[k]
        dJdb[k] = h*(P[k+1]*sigma_div(u))@np.ones((Z.shape[2],1))
    return dJdb


def getYpsilon(Z, omega, mu, eta):
    u = np.transpose(Z[K-1]) @ omega + mu
    return eta(u)

In [ ]:
def getZ(Y0, W, b, K, d, I, h, sigma):
    #initialize Z, Z0=Y0
    Z = np.zeros((K,d,I))
    Z[0] = Y0

    #finds Zk
    for k in range(1,K):
        u = W[k-1]@Z[k-1] + b[k-1]
        Z[k] = Z[k-1] + h*sigma(u)

    return Z

def getP(Z, ypsilon, c, omega, mu, W, b, K, d, I, h, sigma_div, eta_div):
    #initialize P
    P = np.zeros((K+1,d,I))

    #finds P_K
    u = np.transpose(Z[-1])@omega + mu
    P[-1]= np.outer(omega,(ypsilon-c)*eta_div(u))

    #finds P_K-1 to P_0
    for k in range(K-1,0,-1):
        s = W[k]@Z[k] + b[k]
        P[k]=P[k+1] + h*np.transpose(W[k])@(sigma_div(s)*P[k+1])
        
    return P

In [ ]:
a = np.array([[1,2,3],[4,5,6]])
b = np.array([[4,5,6]])
#print(a@b)
#print(np.transpose(a)@b)
#print(a@np.transpose(b))
print(np.outer(a,np.transpose(b)))
"""temp = np.zeros((d,I))
        for i in range(d0):
            temp[i] = Y0[i]
        Y0 = temp"""

#print(a)
#a.resize(4,3)
#print(a)


In [ ]:
def trainingAlgorithm(K, d, h, tau, Y0, c, eta, sigma, eta_div, sigma_div, N = 1000):
    d0 = Y0.shape[0]
    I = Y0.shape[1]
    if d0 < d:
        Y0 = np.resize(Y0, (d,I))
    omega = getomega(d)
    mu = getmu()
    W = getW(K,d)
    b = getb(K,d)
    c = np.transpose(c)
    for i in range(N): #Number of trainingseries
        print(i)
        Z = getZ(Y0, W, b, K, d, I, h, sigma)
        ypsilon = getYpsilon(Z, omega, mu, eta)
        P = getP(Z, ypsilon, c, omega, mu, W, b, K, d, I, h, sigma_div, eta_div)
        
        dJdmu = getdJdmu(Z, ypsilon, c, omega, mu, K, eta_div)
        dJdomega = getdJdomega(Z, ypsilon, c, omega, mu, K, eta_div)
        dJdW = getdJdW(P, Z, W, b, K, h, sigma_div)
        dJdb = getdJdb(P, Z, W, b, K, h, sigma_div)
        
        mu = mu - tau*dJdmu
        omega = omega - tau*dJdomega
        W = W - tau*dJdW
        b = b - tau*dJdb
    return mu, omega, W, b

In [ ]:
def scale(x, alpha, beta):
    a = np.amin(x)
    b = np.amax(x)
    
    x_tilde = 1/(b-a)*((b-x)*alpha + (x-a)*beta)
    return x_tilde, a, b

def inverseScale(x_tilde, a, b, alpha, beta):
    return 1/(beta-alpha)*((x_tilde-alpha)*b + (beta-x_tilde)*a)

In [ ]:
def eta(x):
    return x

def eta_div(x):
    return np.ones_like(x) 

def sigma(x):
    return np.tanh(x)

def sigma_div(x):
    return 1 - np.tanh(x)**2

In [ ]:
def F(y):
    return 1/2*y**2

Y0 = np.linspace(-2,2,100)
Y0.resize(1,100)
c = F(Y0)
alpha = 0.2
beta = 0.8

Y0_tilde, aY0, bY0 = scale(Y0, alpha, beta)
c_tilde, ac, bc = scale(Y0, alpha, beta)


K = 10
d = 2
I = 100
h = 0.1
tau = 0.08


mu, omega, W, b = trainingAlgorithm(K, d, h, tau, Y0_tilde, c_tilde, eta, sigma, eta_div, sigma_div)

In [ ]:
Y_test = np.random.uniform(-2,2,(1,100))

c_test = F(Y_test)


YTest_tilde, aYTest, bYTest = scale(Y_test, alpha, beta)
cTest_tilde, acTest, bcTest = scale(c_test, alpha, beta)

ZTest = getZ(np.resize(YTest_tilde, (2,100)), W, b, K, d, I, h, sigma)
print(ZTest)
ypsilonTest = getYpsilon(ZTest, omega, mu, eta)


ypsilonTest_new = inverseScale(ypsilonTest, acTest, bcTest, alpha, beta)


plt.figure()
plt.plot(Y_test, ypsilonTest_new, label = "test")
plt.plot(Y_test[1], c_test, label = "fasit")
plt.legend()
